In [1]:
import numpy as np # To perform linear algebra
import pandas as pd # For data processing

In [2]:
animes = pd.read_csv("anime-dataset-2023.csv")

In [3]:
animes.head(3)

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,Premiered,Status,Producers,Licensors,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26,"Apr 3, 1998 to Apr 24, 1999",spring 1998,Finished Airing,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41,43,78525,914193,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1,1-Sep-01,UNKNOWN,Finished Airing,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189,602,1448,206248,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26,"Apr 1, 1998 to Sep 30, 1998",spring 1998,Finished Airing,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328,246,15035,356739,727252,https://cdn.myanimelist.net/images/anime/7/203...


In [4]:
animes.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')

In [5]:
animes=animes[['anime_id', 'Name', 'English name', 'Score', 'Genres', 'Synopsis', 'Type', 'Studios', 'Source']]

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer(max_features=5000,stop_words='english') # Create a vector to be used for cosine similarity

In [21]:
#new = pd.concat([animes['anime_id'], animes['Name'], animes['Genres'].str.get_dummies(sep=','), animes['Type'].str.get_dummies(sep=','), animes['Source'].str.get_dummies(sep=',')], axis=1)
#anime_features = new.loc[:, "Adventure":].copy()
new = animes.drop(columns=['Score'])
new['tags'] = new['Genres']
new['tags']

0               Action, Award Winning, Sci-Fi
1                              Action, Sci-Fi
2                   Action, Adventure, Sci-Fi
3        Action, Drama, Mystery, Supernatural
4            Adventure, Fantasy, Supernatural
                         ...                 
24900          Comedy, Fantasy, Slice of Life
24901              Action, Adventure, Fantasy
24902      Action, Adventure, Fantasy, Sci-Fi
24903                                 UNKNOWN
24904                                 UNKNOWN
Name: tags, Length: 24905, dtype: object

In [22]:
vector = cv.fit_transform(new['tags']).toarray() # Transform the tags to be used as a vector
similarity = cosine_similarity(vector)

In [23]:
def recommend(anime):
    index = new[new['Name'] == anime].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:10]:
        print(i)
        print(new.iloc[i[0]].Name)


In [24]:
recommend('Death Note') # Need to apply popularity factors to this model\
print('---')

(2722, 0.9999999999999998)
Death Note: Rewrite
(12808, 0.9999999999999998)
Warau Salesman Special Program
(16682, 0.9999999999999998)
Munou na Nana
(1317, 0.816496580927726)
Shuranosuke Zanmaken: Shikamamon no Otoko
(1720, 0.816496580927726)
Higurashi no Naku Koro ni Kai
(2382, 0.816496580927726)
Shinreigari
(3189, 0.816496580927726)
Higurashi no Naku Koro ni Rei
(3785, 0.816496580927726)
Ga-Rei: Zero
(3858, 0.816496580927726)
Mouryou no Hako
---


In [25]:
recommend('Tokyo Ghoul') # Need to apply popularity factors to this model\
print('---')

(9632, 1.0000000000000002)
Tokyo Ghoul √A
(10513, 1.0000000000000002)
Tokyo Ghoul: "Jack"
(10849, 1.0000000000000002)
Tokyo Ghoul: "Pinto"
(13588, 1.0000000000000002)
Tokyo Ghoul:re
(14405, 1.0000000000000002)
Tokyo Ghoul:re 2nd Season
(15849, 1.0000000000000002)
Gibiate
(276, 0.8660254037844388)
3x3 Eyes
(1346, 0.8660254037844388)
Youma
(5234, 0.8660254037844388)
Gegege no Kitarou: Nippon Bakuretsu
---


In [27]:
recommend('Shingeki no Kyojin') # Need to apply popularity factors to this model\
print('---')

(2469, 0.8944271909999159)
Hidamari no Ki
(4135, 0.8944271909999159)
Birthday Boy
(5873, 0.8944271909999159)
Mahou Shoujo Madoka★Magica
(22594, 0.8944271909999159)
Jinyu
(24089, 0.8944271909999159)
Mida
(980, 0.8451542547285164)
Sentou Yousei Yukikaze
(3271, 0.8451542547285164)
Evangelion: 3.0+1.0 Thrice Upon a Time
(6645, 0.7999999999999999)
Mahou Shoujo Madoka★Magica Movie 3: Hangyaku no Monogatari
(13193, 0.7999999999999999)
Detective Conan Movie 22: Zero the Enforcer
---


In [26]:
import pickle
pickle.dump(new, open('anime_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))